SSL 证书导入

In [1]:
import pip_system_certs.wrapt_requests

建立 LLM 会话  
在报告页找免费 api_key 需要根据模型来源初始化会话

In [2]:
from google import genai

with open(r".\api_key", "r", encoding="utf8") as reader: 
    api_key = reader.readline()
client = genai.Client(api_key=api_key)

model_name = "gemini-2.5-flash"

认证 github

In [3]:
from github.Repository import Repository, NotSet
from github.PullRequest import PullRequest

import github

with open(r".\github_token", "r", encoding="utf8") as reader:
    access_token = reader.readline()

auth = github.Auth.Token(access_token)

筛选 PR

In [4]:
from typing import Literal, List

def filtre(
        repo:Repository,
        pr_state:Literal["open", "close", "all"]=NotSet, # pr 状态
        sort:Literal["created", "updated", "pushed", "popularity"]=NotSet, # 结果排序方式
        direction:Literal["asc", "desc"]=NotSet, # 结果顺序
        base:str=NotSet, # 目标分支
        head:str=NotSet, # 来源分支/来源用户
        prid:int=None, # 指定 pr 编号
        author:str=None, # 指定 pr 发起人
        day_back:int=None, # 回溯指定天数内
    ) -> List[PullRequest]:
    results = repo.get_pulls(pr_state, sort, direction, base, head)
    if prid: results = [pr for pr in results if pr.number == prid]
    if author: results = [pr for pr in results if pr.user.login == author]
    if day_back:
        # datetime 范围内
        from datetime import datetime, timedelta, timezone
        now = datetime.now().replace(tzinfo=timezone.utc)
        start_from = now - timedelta(days=day_back) 
        results = [pr for pr in results if pr.created_at >= start_from]
    return results

In [5]:
## https://github.com/{browser-use/browser-use}/pull/xxx.diff
repository = "browser-use/browser-use"

with github.Github(auth=auth) as g:
    repo = g.get_repo(repository)
    prs = filtre(repo, "open", day_back=2) ## 假设在内部，则用 author + day_back 可以查到指定用户的 PR 这里随便选一些。。
    if not prs: raise ValueError("No matching PR found")

prompt 模板 需要调整

In [6]:
# user_prompt = """Please briefly summarize the changes made to the code from the following diff:
# """
user_prompt = """请简短总结以下的 PR 描述和代码差异，并用数字编号输出差异条目：
PR 描述：
{desc}
代码差异：
{diffs}
"""

In [7]:
from utils import file_io_utils as fiu
from tqdm.notebook import tqdm

def get_llm_resp(query:str):
    return client.models.generate_content(
        model=model_name,
        contents=[query]
    )

def get_llm_diff_resp(pr):
    diffs = fiu.get_remote_contents(pr.diff_url)
    return get_llm_resp(user_prompt.format(desc=f"{pr.title}\n{pr.body}", diffs=diffs))
    
responses = [get_llm_diff_resp(pr) for pr in tqdm(prs)]

  0%|          | 0/5 [00:00<?, ?it/s]

In [8]:
summary_prompt = """请根据以下代码差异描述，总结出一份简报，要求根据日期排序："""
for i, resp in enumerate(responses[::-1]): # 从旧到新
    summary_prompt += f"\n\n{prs[::-1][i].created_at}:\n\n{resp.text}\n"

In [9]:
response = get_llm_resp(summary_prompt)

In [10]:
from IPython.display import Markdown, display

display(Markdown(response.text))

好的，根据您提供的代码差异描述，以下是按日期排序的简报：

---

**简报：近期代码更新摘要**

**日期：2025年9月10日**

*   **通用JavaScript执行工具上线 (04:50:59):**
    *   **内容:** 引入了基于 CDP `Runtime.evaluate` 的通用 JavaScript 执行工具。
    *   **目的:** 旨在解决现有工具难以处理的复杂浏览器交互和数据提取问题，支持坐标点击、表单填充、悬停、拖拽、链接抓取和页面探索等功能。
    *   **影响:** 新增 `ExecuteCDPAction` 模型和 `execute_js` 异步动作，增强了浏览器自动化能力。

*   **`ad-use` README 文档更新 (18:51:55):**
    *   **内容:** 更新了 `ad-use` 项目的 `README.md` 文件。
    *   **目的:** 提高文档清晰度和准确性，并移除过时信息，包括更新警告样式、示例URL（从 `iphone-16-pro` 到 `iphone-17-pro`）及移除旧页脚。

*   **README Python 示例导入顺序统一 (22:05:07):**
    *   **内容:** 统一了 README Python 示例中的导入顺序。
    *   **目的:** 通过对导入进行分组并将 `load_dotenv()` 移至导入语句下方，提高代码清晰度和一致性，遵循最佳实践。

**日期：2025年9月11日**

*   **`news-use` 示例文档链接修复 (05:21:52):**
    *   **内容:** 修复了 `news-use` 示例的文档链接缺失问题。
    *   **目的:** 确保该示例能在“Examples > Apps”分类下显示并可通过侧边栏访问，提升用户体验。

*   **引入验证码看门狗 (CaptchaWatchdog) 支持 (09:35:41):**
    *   **内容:** 新增了验证码看门狗功能。
    *   **目的:** 在验证码求解器启动时自动暂停代理，完成后恢复代理，显著提高在受验证码保护网站上的操作可靠性。
    *   **影响:** 修改了代理执行循环、会话状态管理，并新增 `CaptchaWatchdog` 模块，监听 CDP 事件以控制代理的暂停与恢复。

---